In [194]:
!pwd

/home/tucker/sabbatical/predict_bert_embeddings/char_lm/char_seq_to_masked_emb


In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint, RunOne
from yaml import safe_load
import torch.nn as nn

In [2]:
exphash = 'd40e3553d'
exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

ds = InitDataset(h)


In [3]:
#h['hyperparameters']['manual_attention'] = True
model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[20:59:09] loading configuration file bert_distil_uncased/config.json


[20:59:09] loading weights file bert_distil_uncased/pytorch_model.bin


[20:59:11] Loaded model: <All keys matched successfully>


In [4]:
h['dataset_split'][1] = .01

In [5]:
val_loader = DataLoader(ds[1], batch_size=64, shuffle=False)


In [6]:
ds[0][0]['label_ids'].shape

torch.Size([96])

In [7]:
ds[0][0]['label_ids']

tensor([  101,  2682,  1012,  2129,  2434,   999,  1007,  1012,  1998, 14397,
         5740,  2003,  5129,  2011,  2200,  3492,  1006,  1998,  2402,  1007,
         3057,  2040,  2024,  1999,  2293,  2007,  2032,  1012,  2129, 17203,
         1012,  2065,  2017,  2428,  2066,  1996,  1000,  2613,  1000,  2632,
        14397,  5740,  1010,  2123,  1005,  1056,  2130,  2228,  2000,  3422,
         2023,  2143,  1012,  2017,  2453,  2022,  3724,  2000,  2228,  2008,
         4469,  3334, 28533, 14482,  2015,  2031,  2999,  2032,  2007,  1037,
         2919,  6100,  1012,  2632, 14397,  5740,  2003,  2028,  1997,  2216,
         2261,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0])

In [8]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [9]:
xent = nn.CrossEntropyLoss()
def xentropy_loss_fn(output, labels):
  output = output.contiguous()  
  return xent(output.view(-1, output.size(-1)), labels.view(-1))

def top1_acc_fn(output, labels):
  top = output.argmax(-1)
  right = top==labels
  return right.float()

def Validate(val_loader, model):
  device = next(model.parameters()).device
  cum_acc = 0
  cum_xent_loss = 0

  rights_ = []
  inputs_ =[]
  labels_ =[]
  maxes_ =[]
  for i, data in enumerate(val_loader):
    data = {k: d.to(device) for k,d in data.items()}
    inputs = data['input_ids']
    labels = data['label_ids']
    inputs_ += Torch2Py(inputs)
    labels_ += Torch2Py(labels)
    
    with torch.no_grad():
      
      model.eval()
      outputs = model(inputs)
      maxes_ += Torch2Py(outputs.argmax(-1))
      xent_loss = xentropy_loss_fn(outputs, labels)
      cum_xent_loss += xent_loss      

      top1_acc = top1_acc_fn(outputs, labels)
      rights_ += Torch2Py(top1_acc)
      cum_acc += top1_acc.mean()
    if len(inputs_)>100:
      break

  steps = i+1
  print(cum_acc/steps, cum_xent_loss.detach()/steps)
  return rights_, inputs_, labels_, maxes_,



In [10]:
right, inputs, labels, maxes = Validate(val_loader, model)

tensor(0.1104, device='cuda:0') tensor(11.1539, device='cuda:0')


In [11]:
len(right), len(inputs), len(labels), len(maxes),

(128, 128, 128, 128)

In [12]:
model.embedded_chars[0].shape

torch.Size([96, 768])

In [13]:
char_to_idx_map = torch.load('../data/char_to_idx_map2.pt')

In [14]:
idx_to_char_map= {v:k for k,v in char_to_idx_map.items()}

In [15]:
idx_to_tok_map = {}
with open('../data/bert-base-uncased-vocab.txt') as f:
  for i, l in enumerate(f.readlines()):
    idx_to_tok_map[i] = l.strip()

In [16]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [17]:
def GetToks(encoded):
  s = ""
  for c in encoded:
    s+= " " + idx_to_tok_map[c]
  return s

In [18]:
inps = ds[0][0]['input_ids'].unsqueeze(0)

In [19]:
with torch.no_grad():
  a = ((model.embedded_chars[-1, 0:8, :].unsqueeze(1)-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [20]:
model.embedded_chars[-1, 0:20, :].unsqueeze(1).shape

torch.Size([20, 1, 768])

In [21]:
with torch.no_grad():
  a = (model.embedded_chars[-1, 0:20, :].squeeze(1)@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)

In [25]:
print(GetChars(inputs[1]))

[cls] rat-pelts, is [m]. he scarcely looks human, [m] shadow-image of the successful doctor, yukio. [m]tekichi represents the oppression of japan's violent, disordered-past from the eras of the shogun and the samurai[m] yukio represents the emergence of a modern japan, with his work as a doctor and his bourgeois life with his new-wife[m] rin. he to[m] helping cure the______________


In [26]:
GetChars(Torch2Py(ds[0][0]['input_ids']))

'[cls] above. how original!). and pacino is [m] by very pretty (and [m]) girls who are in love [m] him. how pathetic. if you really like the "real[m] al pacino, don\'t even think to watch this film. you might be pushed to think that extraterrestrials [m] replaced him with a bad copy. al pacino is one of [m] few__________________________________________________________________________'

In [27]:
print(GetToks(maxes[1]))

 ) " 7 ) is unknown . he hardly looks human " ) shadow 15 of the young doctor ) ##yuki ) he represents the oppression of japan . violent ) disorder 15 from the birth of the earth and the samurai ##yuki represents the emergence of a modern japan , with his work as a doctor and his bourgeois life with his friend / ##s . he is help cure the [ [ [ [ [ [ , , , ... ... ... ... ... ... ##oh ##oh ##oh ##oh ##oh ##oh ##oh ##oh ##o


In [86]:
j = 0
inp = inputs[4][j:j+18]
GetChars(inp), inp

('[cls] [m](it remin',
 [32, 46, 55, 62, 33, 1, 32, 56, 33, 9, 52, 63, 1, 61, 48, 56, 52, 57])

In [87]:
inp = torch.tensor([[50, 64, 52, 63, 44, 61,  1, 35, 41, 40, 35,  1, 58, 45, 62, 46, 48, 57]])

In [88]:
out = model.char_embedder.tokens_to_emb(inp.cuda())

In [89]:
#next(model.char_embedder.tokens_to_emb.named_parameters())

In [90]:
out[0][:4]

tensor([ 0.0719, -0.1866,  0.1774,  0.5457], device='cuda:0',
       grad_fn=<SliceBackward>)

In [91]:
word = (out@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)

In [92]:
model.bert.distilbert.embeddings.word_embeddings.weight.data[2858][:10]

tensor([ 0.0209, -0.0467, -0.0392, -0.0345,  0.0150, -0.0398, -0.0696, -0.0479,
        -0.0290, -0.0037], device='cuda:0')

In [93]:
word

tensor([2858], device='cuda:0', grad_fn=<NotImplemented>)

In [94]:
word = ((out-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [274]:
w = '[cls] guitars [m] some ot [m]'
thing = []
for c in w :
  thing.append(char_to_idx_map[c])
print(len(thing))
inp = torch.tensor(thing).unsqueeze(0)

29


In [248]:
LoadCheckpoint(model.char_embedder, '../single_emb_pred/models/c26917206')

[22:44:05] Loaded model: <All keys matched successfully>


In [275]:
out = model(inp.cuda())

In [276]:
out = model.embedded_chars

In [239]:
out = model.char_embedder(inp.cuda())

In [240]:
out.shape

torch.Size([1, 96, 768])

In [295]:
.005**2

2.5e-05

In [294]:
torch.nn.MSELoss(reduction='none')(torch.tensor([[.005, 0.0, 0.0]]), torch.tensor([[0.0, 0.0, 0.0]]))

tensor([[2.5000e-05, 0.0000e+00, 0.0000e+00]])

In [280]:
word = ((out[0, :4].unsqueeze(1)-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [278]:
word = (out@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)[0]

In [281]:
GetToks(Torch2Py(word))

' [CLS] guitars [MASK] some'

In [ ]:
' [CLS] made of stories a where every story is great ; that'

In [42]:
a.shape

torch.Size([8])

In [24]:
print(GetToks(Torch2Py(a)))

 [CLS] retired / it doesn in matter if


In [64]:
print(GetToks(Torch2Py(a)))

 comedy at at at at at now at at at at at at at now at at comedy at at


In [ ]:
p

In [25]:
embeddings = ds.data['embeddings']

AttributeError: 'tuple' object has no attribute 'data'

In [26]:
embeddings.norm(2,-1).mean()

NameError: name 'embeddings' is not defined

In [252]:
embedding_noise = .25

In [253]:
r = torch.normal(mean=embedding_noise, std=1, size=embeddings.size()).cuda()
norm_factor = torch.norm(r, 2, -1).mean()/embedding_noise
normalized_noise = r/norm_factor
new = embeddings + normalized_noise

In [254]:
((new-embeddings)**2).mean()

tensor(8.1433e-05, device='cuda:0')

In [255]:
torch.sqrt((new-embeddings)**2).mean()

tensor(0.0072, device='cuda:0')

In [217]:
n = new.norm(2,-1)

In [31]:
def MakeHist(h, name=None):
  y = h[0]
  x= h[1]
  return {
    'name': name,
    'type': 'bar',                                                                                                                   'x': x,
          'marker': dict(opacity=.7),
   'y': y
  }

In [30]:
dat = MakeHist(np.histogram(n, bins=30), name='toks')

fig = fig = go.Figure(data=[dat], layout={'barmode':'overlay'})
plotly.offline.plot(fig, filename='/home/tucker/Downloads/imdb_size_distci.html')


NameError: name 'n' is not defined

In [27]:
attn = torch.nn.Sigmoid()(model.char_embedder.char_block_attn)

In [28]:
hm = go.Heatmap(z=attn.detach().cpu().numpy())
fig = go.Figure(data=[hm])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [29]:
sb = model.char_embedder.switchboard[0]

In [30]:
hm = go.Heatmap(z=sb.detach().cpu().numpy())
fig = go.Figure(data=[hm])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [76]:
ds = CharClozeToDenseTokEmbDataset('imdb_tok96_char_encoded384.pt', num_masks=20)

In [77]:
char_to_idx_map = torch.load('../data/char_to_idx_map.pt')

In [78]:
idx_to_char_map= {v:k for k,v in char_to_idx_map.items()}

In [82]:
idx_to_tok_map = {}
with open('../data/bert-base-uncased-vocab.txt') as f:
  for i, l in enumerate(f.readlines()):
    idx_to_tok_map[i] = l.strip()

In [83]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [89]:
def GetToks(encoded):
  s = ""
  for i, c in enumerate(encoded):
    s+= " %d."%i + idx_to_tok_map[c]
  return s

In [27]:
print(GetChars(Torch2Py(ds[0]['input_ids'])))
print(GetChars(Torch2Py(ds[0]['original'])))
print(GetToks(Torch2Py(ds[0]['tokens'])))

TypeError: list indices must be integers or slices, not str

In [61]:
ds[0] 

tensor(76)
tensor([[ 72,  76],
        [ 92,  95],
        [ 96, 101],
        [114, 115],
        [116, 122],
        [122, 123],
        [126, 130],
        [137, 138],
        [139, 142],
        [145, 150],
        [162, 163],
        [168, 171],
        [192, 196],
        [209, 213],
        [220, 221],
        [227, 233],
        [234, 241],
        [252, 253],
        [253, 254],
        [269, 272]], dtype=torch.int16)


{'input_ids': tensor([34, 42, 51, 58, 36,  1, 48,  1, 40, 43, 52, 48, 59, 13,  1, 59, 47, 44,
          1, 46, 57, 44, 40, 59,  1, 52, 40, 49, 54, 57, 48, 59, 64,  1, 54, 45,
          1, 45, 48, 51, 52, 58,  1, 57, 44, 51, 44, 40, 58, 44, 43,  1, 41, 44,
         45, 54, 57, 44,  1, 58, 40, 64,  1, 18, 26, 20, 20,  1, 40, 57, 44,  1,
         34, 38, 36,  1, 53, 54, 59,  1, 45, 54, 57,  1, 52, 44, 15,  1, 54, 45,
          1, 34, 38, 36,  1, 34, 38, 36,  1, 54, 57,  1, 58, 54,  1,  3, 52, 40,
         49, 54, 57, 34, 38, 36,  1, 34, 38, 36, 34, 38, 36,  1, 48,  1, 34, 38,
         36,  1, 61, 48, 44, 62, 44, 43, 34, 38, 36,  1, 34, 38, 36,  1, 48,  1,
         34, 38, 36,  1,  9, 59, 47, 44,  1, 42, 57, 54, 62, 43, 10, 34, 38, 36,
          1, 40, 53, 43,  1, 34, 38, 36,  1, 62, 44, 57, 44,  1, 61, 44, 57, 64,
          1, 46, 54, 54, 43,  1,  9, 59, 47, 44,  1, 34, 38, 36,  1, 42, 54, 52,
         52, 40, 53, 43,  1, 40, 53, 43,  1, 34, 38, 36,  1, 51, 48, 46, 47, 59,
         58, 34

In [7]:
ds.data['token_start_offsets'].shape

torch.Size([200091, 96, 2])

In [39]:
original = ds.data['chars_encoded'][3]
offsets = ds.data['token_start_offsets'][3]
tokens = ds.data['token_ids'][3]
num_toks = (tokens!=0).sum()
rand_toks = torch.randperm(num_toks)
offsets = offsets[rand_toks[:ds.num_masks]]
sorted_offsets = torch.sort(offsets, 0)[0]
segments_to_keep = torch.cat([ds.beg, sorted_offsets.view(-1), ds.end], 0).view(-1,2)

slices = []
char_mask = torch.tensor([34,38,36]).char().to('cuda')
for start, end in segments_to_keep:
  char_slice = original[start:end]
  slices.append(char_slice)
  slices.append(char_mask)

masked_seq = torch.cat(slices, 0)
pad_size = max(0, len(original)-len(masked_seq))
pad = torch.tensor([0]).expand(pad_size).char().to('cuda')
masked_seq = torch.cat([masked_seq, pad], 0)[:len(original)]

5

In [22]:
pad_size

5

In [322]:
original ds.data['token_start_offsets'][0]

AttributeError: 'Tensor' object has no attribute 'randperm'

In [1]:
import torch

In [2]:
a = torch.tensor([[.1,.2,.3,.4,.5]])
b = torch.tensor([[.5,.4,.3,.2,.1]])
c = torch.tensor([[.1,.1,.1,.1,.5]])

In [4]:
def nll(teacher, student):
  return -1*(teacher*torch.log(student)).sum()

In [9]:
nll(b,c)

tensor(3.2929)

In [89]:
class StepWithSigmoidGrad(torch.autograd.Function):
    @staticmethod
    def forward(self,x):
        self.x = x
        return (x > 0).float()

    @staticmethod
    def backward(self,grad_output):
        sig = torch.nn.Sigmoid()(self.x)
        return sig*(1-sig)

class StepWithHardGrad(torch.autograd.Function):
    @staticmethod
    def forward(self,x):
        self.high = (x>-.5) & (x<.5)
        return (x > 0).float()

    @staticmethod
    def backward(self,grad_output):
        grad_input = grad_output.clone()
        print(self.high)
        grad_input[self.high] = 1
        grad_input[~self.high] = 0
        return grad_input
      

In [96]:
f = StepWithHardGrad

In [97]:
vals = []
for i in torch.arange(-2,2,.1):
  t = i.clone().detach()
  t.requires_grad =True
  v = f.apply(t)
  v.backward()
  vals.append((t.item(), v.item(), t.grad.item()))

tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)


In [98]:
vals

[(-2.0, 0.0, 0.0),
 (-1.899999976158142, 0.0, 0.0),
 (-1.7999999523162842, 0.0, 0.0),
 (-1.7000000476837158, 0.0, 0.0),
 (-1.600000023841858, 0.0, 0.0),
 (-1.5, 0.0, 0.0),
 (-1.399999976158142, 0.0, 0.0),
 (-1.2999999523162842, 0.0, 0.0),
 (-1.2000000476837158, 0.0, 0.0),
 (-1.100000023841858, 0.0, 0.0),
 (-1.0, 0.0, 0.0),
 (-0.8999999761581421, 0.0, 0.0),
 (-0.800000011920929, 0.0, 0.0),
 (-0.699999988079071, 0.0, 0.0),
 (-0.6000000238418579, 0.0, 0.0),
 (-0.5, 0.0, 0.0),
 (-0.4000000059604645, 0.0, 1.0),
 (-0.30000001192092896, 0.0, 1.0),
 (-0.20000000298023224, 0.0, 1.0),
 (-0.10000000149011612, 0.0, 1.0),
 (0.0, 0.0, 1.0),
 (0.10000000149011612, 1.0, 1.0),
 (0.20000000298023224, 1.0, 1.0),
 (0.30000001192092896, 1.0, 1.0),
 (0.4000000059604645, 1.0, 1.0),
 (0.5, 1.0, 0.0),
 (0.6000000238418579, 1.0, 0.0),
 (0.699999988079071, 1.0, 0.0),
 (0.800000011920929, 1.0, 0.0),
 (0.8999999761581421, 1.0, 0.0),
 (1.0, 1.0, 0.0),
 (1.100000023841858, 1.0, 0.0),
 (1.2000000476837158, 1.0, 0.0),

In [99]:
t,v,g = zip(*vals)

In [100]:
dat1 = {'type':'scatter', 'x':t, 'y':v}
dat2 = {'type':'scatter', 'x':t, 'y':g}

In [101]:
fig = fig = go.Figure(data=[dat1,dat2])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/imdb_size_distci.html')


'/home/tucker/Downloads/imdb_size_distci.html'